In [1]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import time
import numpy as np
import ast

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener el valor de OPENAI_API_KEY
api_key = os.getenv("OPENAI_API_KEY")

gpt_client = OpenAI(
    api_key = api_key,
)

# Carga el archivo Parquet
data = pd.read_parquet("../../supermercados/plazavea/plazavea.parquet")

In [ ]:
# Función actualizada para obtener embeddings
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    response = gpt_client.embeddings.create(input = [text], model=model).data[0].embedding
    return response

# Generar embeddings y agregarlos como una nueva columna
data["embedding"] = data.apply(
    lambda row: get_embedding(
        f"Producto: {row['product_name']}. Descripción: {row['descripcion_producto']}. Categorías: {' ,'.join(row['categories'])}"
    ), 
    axis=1
)

In [4]:
data['categories_txt'] = data['categories'].apply(lambda x: ', '.join(ast.literal_eval(x) if isinstance(x, str) else x))


# Guarda los embeddings para uso posterior
data.to_parquet("productos_con_embeddings.parquet")